# MBOGO JOSEPH MBOTE
## SCT211-0011/2018

In [1]:
import os
import webbrowser
import folium
import json
import requests
import uuid

from tkinter import *
from tkinter import messagebox
from geopy import distance, Nominatim

from Points import Points
from Graph import Graph
from Algorithms import A_Star, BFS, DFS, RP

In [2]:
geocoder = Nominatim(user_agent="search")
_, KE = geocoder.geocode("Kenya")
KE = list(KE)
global leaflet_map
leaflet_map = folium.Map(location=KE, tiles='OpenStreetMap', zoom_start=6)

In [3]:
initial_state = ['Eldoret' , 'Garissa' , 'Isiolo', 'Kisumu', 'Kitui', 'Malindi' , 'Mombasa' , 'Nairobi' , 'Nakuru' , 'Thika']
terminal_state = ['Eldoret' , 'Garissa' , 'Isiolo', 'Kisumu', 'Kitui', 'Malindi' , 'Mombasa' , 'Nairobi' , 'Nakuru' , 'Thika']
algorithms = ['A* Search','Breadth First Search', 'Depth First Search']
town_locations = dict()

In [4]:
def line_visualizer(path, map_to_draw_on=leaflet_map):
    points = list(map(lambda n: town_locations[n], path))
    folium.PolyLine(points,color="purple").add_to(map_to_draw_on)

In [5]:
def point_generation():
    for town in initial_state:
        _, loc = geocoder.geocode(f"{town}, Kenya")
        loc = list(loc)
        town_locations[town] = loc
        folium.Marker(location = loc,popup = town,icon=folium.Icon(color="purple")).add_to(leaflet_map)
point_generation()

In [6]:
eldoret = Points('Eldoret')
garissa = Points('Garissa')
isiolo = Points('Isiolo')
kisumu = Points('Kisumu')
kitui = Points('Kitui')
malindi = Points('Malindi')
mombasa = Points('Mombasa')
nairobi = Points('Nairobi')
nakuru = Points('Nakuru')
thika = Points('Thika')


graph = Graph()
graph.addVertex(eldoret)
graph.addVertex(garissa)
graph.addVertex(isiolo )
graph.addVertex(kisumu)
graph.addVertex(kitui)
graph.addVertex(malindi)
graph.addVertex(mombasa)
graph.addVertex(nairobi)
graph.addVertex(nakuru)
graph.addVertex(thika)


graph.addDirectedEdge(kisumu, eldoret)
graph.addDirectedEdge(eldoret, kisumu)
graph.addDirectedEdge(kisumu, nakuru)
graph.addDirectedEdge(nakuru, kisumu)
graph.addDirectedEdge(eldoret, nakuru)
graph.addDirectedEdge(nakuru, eldoret)
graph.addDirectedEdge(eldoret, nairobi)
graph.addDirectedEdge(nairobi, eldoret)
graph.addDirectedEdge(isiolo, thika)
graph.addDirectedEdge(thika, isiolo)
graph.addDirectedEdge(nakuru, nairobi)
graph.addDirectedEdge(nairobi, nakuru)
graph.addDirectedEdge(nairobi, thika)
graph.addDirectedEdge(thika, nairobi)
graph.addDirectedEdge(nairobi, mombasa)
graph.addDirectedEdge(mombasa, nairobi)
graph.addDirectedEdge(kitui, mombasa)
graph.addDirectedEdge(mombasa,kitui)
graph.addDirectedEdge(nairobi, garissa)
graph.addDirectedEdge(garissa,nairobi)
graph.addDirectedEdge(nairobi,isiolo)
graph.addDirectedEdge(nairobi,isiolo)
graph.addDirectedEdge(garissa,isiolo)
graph.addDirectedEdge(isiolo,garissa)
graph.addDirectedEdge(isiolo,kitui)
graph.addDirectedEdge(kitui,isiolo)
graph.addDirectedEdge(garissa,kitui)
graph.addDirectedEdge(kitui,garissa)
graph.addDirectedEdge(nairobi,kitui)
graph.addDirectedEdge(kitui,nairobi)
graph.addDirectedEdge(kitui,malindi)
graph.addDirectedEdge(malindi,kitui)
graph.addDirectedEdge(garissa,malindi)
graph.addDirectedEdge(malindi,garissa)
graph.addDirectedEdge(mombasa,malindi)
graph.addDirectedEdge(malindi,mombasa)

In [7]:
def town_position(town_name, towns):
    return graph.vertices.index(list(filter(lambda n: n.value == town_name, towns.vertices))[0])

In [8]:
def obtain_distance(path):
    global total_distance
    total_distance=0
    for i in range(len(path)-1):
        x = path[i]
        y = path[i+1]
        _, x_loc = geocoder.geocode(x)
        _, y_loc = geocoder.geocode(y)
        resp=requests.get(f'http://router.project-osrm.org/route/v1/driving/{x_loc[1]},{x_loc[0]};{y_loc[1]},{y_loc[0]}?overview=false')
        distance=((json.loads(resp.content))["routes"][0]["distance"])*0.001
        total_distance+=distance 
    return round(total_distance, 2)

In [9]:
def interaction():
    filename = str(uuid.uuid4())
    print(filename)
    global algorithm,initial,terminal,route_button
    print(algorithm.get(),initial.get(),terminal.get())
    
    
    if "A* Search" == algorithm.get():
        came_from, cost = A_Star(graph, graph.vertices[town_position(terminal.get(), graph)], town_position(initial.get(), graph))
        print(cost)
        path = RP(came_from, graph.vertices[town_position(initial.get(), graph)].value, graph.vertices[town_position(terminal.get(), graph)].value)
        print(path)
        line_visualizer(list(filter(lambda n: not isinstance(n, int), path)))
        route_button.config(state="disabled")
        leaflet_map.save(f"{filename}.html")
        messagebox.showinfo('DISTANCE',"Total Distance = " + str(obtain_distance(path)))
        url = f"file:///C:/Users//KBS-Assignments/Assignment 2/{filename}.html"
        window.destroy()
        os.system('python Gensearch.py')
    
    
    elif "Breadth First Search" == algorithm.get():
        path = BFS(graph.vertices[town_position(terminal.get(), graph)], graph, town_position(initial.get(), graph))
        print(path)
        line_visualizer(list(map(lambda n: n.value, path)))
        route_button.config(state="disabled")
        leaflet_map.save(f"{filename}.html")
        messagebox.showinfo('DISTANCE', "Total Distance = " + str(obtain_distance(path)))
        url = f"file:///C:/Users//KBS-Assignments/Assignment 2/{filename}.html"
        window.destroy()
        os.system('python Gensearch.py')
        
        
    else:
        path = DFS(graph.vertices[town_position(terminal.get(), graph)], graph, town_position(initial.get(), graph))
        line_visualizer(list(map(lambda n: n.value, path)))
        route_button.config(state="disabled")
        leaflet_map.save(f"{filename}.html")
        messagebox.showinfo('DISTANCE', "Total Distance = " + str(obtain_distance(path)))
        url = f"file:///C:/Users//KBS-Assignments/Assignment 2/{filename}.html"
        window.destroy()
        os.system('python Gensearch.py')           

In [15]:
window = Tk() 
window.title("Mbote Joseph Route Finder")
window.geometry("1350x750")
window.maxsize(1350,750)
window.minsize(1350,750)
window.configure(bg='#041b2d') 


img = PhotoImage(file = "Kenya.png")
image_label = Label(window,image=img,bg='#041b2d')
image_label.place(x=330, y=85)


algorithm = StringVar(window)
algorithm.set("Select Strategy: ")
algorithm_menu = OptionMenu(window,algorithm,"A* Search", "Breadth First Search", "Depth First Search")
algorithm_menu.config(width=18, fg="#041b2d",bg="white", activebackground="#65F3B9")
algorithm_menu.place(x=685,y=225)


initial = StringVar(window)
initial.set('Select Town: ')
initial_menu = OptionMenu(window, initial, 'Eldoret' , 'Garissa' , 'Isiolo', 'Kisumu', 'Kitui', 'Malindi' , 'Mombasa' , 'Nairobi' , 'Nakuru' , 'Thika')
initial_menu.config(width=12, fg="#041b2d",bg="white", activebackground="#65F3B9")
initial_menu.place(x=615, y=335)


terminal = StringVar(window)
terminal.set('Select Town: ')
terminal_menu = OptionMenu(window, terminal,'Eldoret' , 'Garissa' , 'Isiolo', 'Kisumu', 'Kitui', 'Malindi' , 'Mombasa' , 'Nairobi' , 'Nakuru' , 'Thika')
terminal_menu.config(width=12, fg="#041b2d",bg="white", activebackground="#65F3B9")
terminal_menu.place(x=800, y=335)


route_button = Button(window,text="OBTAIN ROUTE", fg="white",bg="#041b2d", activebackground="#ABDEE6" ,font=("Calibri",13,"bold"),bd=5,command=interaction)
route_button.place(x=695,y=400)


restart_button = Button(window,text="RESTART", fg="#041b2d",bg="white", activebackground="#ABDEE6" ,font=("Calibri",13,"bold"),bd=5, command=window.destroy)
restart_button.place(x=645,y=580)

window.mainloop()

51b2e529-2947-41be-a36a-ae602fa526f6
Depth First Search Kisumu Malindi


2022-07-27 10:44:11.498 python[48254:1367125] Warning: Expected min height of view: (<NSButton: 0x7fd4bae37780>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2022-07-27 11:07:40.365 python[48254:1367125] Warning: Expected min height of view: (<NSButton: 0x7fd4b9076e20>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
python: can't open file '/Users/user/Desktop/Assignment 2 - Search Strategies/Gensearch.py': [Errno 2] No such file or directory
